In [1]:
import sys
import re
import pandas as pd
from typing import List, Dict
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
metadata = pd.read_csv('metadata.csv')
author = metadata['author']
display(author)

/tmp/ipykernel_5943/419966214.py:1: DtypeWarning: Columns (1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('metadata.csv')


0                                 Anonymous
1                    Seneca, Lucius Annaeus
2                     Hodgson, William Hope
3                     Waddington, Mary King
4             Lindsay, Anna Robertson Brown
                        ...                
72214                       Betham, Matilda
72215    Bradford, Sarah H. (Sarah Hopkins)
72216                       Dante Alighieri
72217                   Douglass, Frederick
72218                      Lincoln, Abraham
Name: author, Length: 72219, dtype: object

In [9]:
metadata = pd.read_csv('metadata.csv')
author = metadata['author']
f_name = []
m_name = []
for row in author.values:
    if pd.notna(row):
        row = str(row)
        if "," in row:
            parts = row.split(", ")
            name = parts[1] + " " + parts[0]
            if '(' in name:
                split_parts = name.split(' (', 1)
                First = split_parts[0]
                ml = split_parts[-1]
                parts = ml.split(') ',1)
                Middle = parts[0]
                Last = parts[-1]
                fn = First + Last
                mn = Middle + " " + Last
                f_name.append(fn)
                m_name.append(mn)
            else:
                f_name.append(name)
                m_name.append("empty")
        else:
            f_name.append(row)
            m_name.append("empty")
    else:
        f_name.append('empty')
        m_name.append('empty')
        
metadata['F_name'] = f_name
metadata['M_name'] = m_name
metadata.to_csv('update_metadata.csv')

names = metadata[['F_name','M_name']]
names = names.drop_duplicates(subset='F_name')

display(names)


# def remove_parentheses(text):
#     return re.sub(r'\([^)]*\)\s*', '', text)

# name = [remove_parentheses(s) for s in a_name]

# name_df = pd.DataFrame({'Author_Name': name})
# name_df = name_df[(name_df['Author_Name'] != "Anonymous") | (name_df['Author_Name'] != "Various")]
# display(name_df)

/tmp/ipykernel_5943/2979316675.py:1: DtypeWarning: Columns (1,9) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('metadata.csv')


,F_name,M_name
0,Anonymous,empty
1,Lucius Annaeus Seneca,empty
2,William Hope Hodgson,empty
3,Mary King Waddington,empty
4,Anna Robertson Brown Lindsay,empty
...,...,...
72184,Wilton W.Blancké,Wilton Wallace Blancké
72189,Charles Inman Barnard,empty
72191,Karl Gutzkow,empty
72200,Virginia Carter Castleman,empty


In [4]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Mozilla/5.0")
print("connected")
def getresult(query, count):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent="Mozilla/5.0")
    sparql.addExtraURITag("timeout","10000")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    results = results['results']['bindings']
    if results:
        new_results = []
        for result in results:
            new_result = {}
            for key in result:
                new_result[key] = result[key]['value']
            new_results.append(new_result)
        return new_results
    else:
        count += 1

connected


In [5]:
query = """
SELECT ?item ?itemLabel ?work ?workLabel
WHERE {
  ?item rdfs:label "J. K. Rowling"@en.
  ?item wdt:P106 ?occupation;
        wdt:P800 ?work.
  FILTER(?occupation = wd:Q482980 || ?occupation = wd:Q36180) # Q482980 is the QID for 'author'.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
p = getresult(query,0)
result_df = pd.DataFrame.from_dict(p)
print(type(result_df))
display(result_df)

<class 'pandas.core.frame.DataFrame'>


,item,work,itemLabel,workLabel
0,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q8337,J. K. Rowling,Harry Potter
1,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q43361,J. K. Rowling,Harry Potter and the Philosopher's Stone
2,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q46751,J. K. Rowling,Harry Potter and the Goblet of Fire
3,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q46758,J. K. Rowling,Harry Potter and the Deathly Hallows
4,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q46887,J. K. Rowling,Harry Potter and the Half-Blood Prince
5,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q47209,J. K. Rowling,Harry Potter and the Chamber of Secrets
6,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q47598,J. K. Rowling,Harry Potter and the Prisoner of Azkaban
7,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q80817,J. K. Rowling,Harry Potter and the Order of the Phoenix
8,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q8337,J. K. Rowling,Harry Potter
9,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q43361,J. K. Rowling,Harry Potter and the Philosopher's Stone


In [7]:
#test = names.iloc[16000:]
base_query = """
SELECT ?item ?itemLabel ?work ?workLabel
WHERE {
  ?item rdfs:label "%s"@en.
  ?item wdt:P106 ?occupation;
        wdt:P800 ?work.
  FILTER(?occupation = wd:Q482980 || ?occupation = wd:Q36180) # Q482980 is the QID for 'author'.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""
count = 0
for index, row in metadata.iterrows():
    author = row['M_name']
    if pd.notna(author):
        print(author)
        if (author != "empty") and pd.notna(author):
            author = author.split('"')[0]
            formatted_query =  base_query % (author)
            df_1 = getresult(formatted_query, count)
            if df_1 is not None:
                rdf = pd.DataFrame.from_dict(df_1)
                result_df = pd.concat([result_df,rdf])
print(count)
display(result_df)

empty
empty
empty
empty
empty
empty
empty
empty
empty
Samuel H. Hammond
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Charles Stephen Brooks
Frederick J. Cross
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Elsie Riach Murray
empty
empty
Harrie Irving Hancock
empty
empty
empty
Margaret Oliphant
empty
Margaret Oliphant
Margaret Oliphant
Margaret Oliphant
empty
empty
empty
empty
empty
empty
Lyman Frank Baum
empty
empty
empty
Henry Francis Keenan
empty
empty
James Montgomery Beck
empty
empty
empty
empty
empty
empty
Stephen Meredyth Edwardes
empty
Joseph Morris Bachelor
empty
empty
empty
empty
empty
empty
empty
empty
Clair Wallace Hayes
Arthur John Rees
empty
empty
empty
empty
empty
empty
Donald Alexander Mackenzie
empty
empty
empty
empty
empty
empty
empty
empty
empty
William Thomas Massey
empty
empty
empty
empty
William Henry Hudson
empty
empty
empty
empty
empty
empty
empty
John 

empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Robert Michael Ballantyne
empty
empty
John Rogers Bolles
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
William Sanday
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
David Leslie Murray
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Charles Waddell Chesnutt
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
James Allanson Picton
Alice Muriel Williamson
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Daniel Garrison Brinton
empty
Harriet Ann Jacobs
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
em

empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
James Matthew Barrie
empty
empty
Robert William Chambers
empty
empty
empty
empty
empty
empty
empty
empty
Grace Smith Richmond
Henry Rider Haggard
empty
Thornton Waldo Burgess
empty
empty
empty
empty
empty
empty
empty
empty
John Lyth
William George Archer
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Caroline Frances Eleanor Spurgeon
Charlotte Bronte Herr
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Joseph Alexander Altsheler
empty
empty
empty
Agnes Maude Royden
John Christian Symons
empty
empty
empty
empty
empty
William Wymark Jacobs
William Wymark Jacobs
William Wymark Jacobs
William Wymark Jacobs
William Wymark Jacobs
William Wymark Jacobs
William Stewart Wallace
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
emp

Harrie Irving Hancock
empty
empty
Harrie Irving Hancock
Harrie Irving Hancock
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Herbert George Wells
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Edward Alfred Martin
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Harrie Irving Hancock
Harrie Irving Hancock
Harrie Irving Hancock
Harrie Irving Hancock
Harrie Irving Hancock
empty
empty
empty
empty
empty
empty
empty
empty
Philip Richard Thornhagh Gurdon
empty
empty
empty
empty
Justin Dewey Fulton
empty
empty
empty
empty
empty
empty
empty
empty
Samuel Dickey Gordon
empty
empty
empty
empty
empty
empty
empty
Clair Wallace Hayes
Harrie Irving Hancock
Harrie Irving Hancock
empty
Samuel Dickey Gordon
empty
Harrie Irving Hancock
empty
empty
empty
empty
empty
empty
empty
empty
Harrie Irving Hancock
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Louis John Stellman
em

empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Samuel Rutherford Crockett
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Edward Hamilton Currey
Henry Rider Haggard
empty
empty
empty
Robert Daniel Carmichael
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Edward Reynolds Pease
empty
empty
Edwin Monroe Bacon
empty
empty
empty
empty
empty
empty
empty
Herbert Lord Havell
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Robert Henry Elliot
empty
empty
empty
em

...

empty
empty
Alfred Hugh Fisher
empty
empty
empty
Peter Lund Simmonds
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Horace Bowne Fyfe
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Roger Thompson Finlay
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
William Livingston Alden
empty
Stanton Arthur Coblentz
empty
empty
empty
Edward Gould Buffum
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Paul Adriaan Daum
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
William Crispin Sheppard
empty
empty
Eleanor Catherine Price
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty

empty
empty
George Alfred Henty
empty
empty
empty
empty
empty
empty
empty
William Henry Hudson
empty
William Harris Wharton
George Harvey Ralphson
empty
empty
empty
empty
empty
empty
empty
empty
empty
Alan Alexander Milne
empty
empty
empty
empty
empty
empty
empty
empty
empty
Francis Marion Crawford
empty
empty
empty
Charlotte Mary Yonge
empty
empty
empty
empty
empty
empty
empty
empty
William Matthew Flinders Petrie
Charlotte Mary Yonge
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
William Henry Hudson
empty
empty
empty
empty
empty
empty
empty
empty
John Caldwell Calhoun
empty
empty
empty
William Matthew Flinders Petrie
empty
William Henry Hudson
empty
empty
empty
empty
empty
empty
empty
empty
Pelham Grenville Wodehouse
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Richard Doddridge Blackmore
Maria Louise Greene
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
William Henry Hudson
empty
William Butler

Edith Nesbit
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Mary Elizabeth Wilson Sherwood
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Richard Doddridge Blackmore
empty
empty
empty
Ida Treadwell Thurston
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Edith Hedden Allen
empty
empty
empty
empty
empty
empty
Rudolf Chambers Lehmann
Lily Adams Beck
empty
empty
empty
empty
empty
empty
empty
Robert William Chambers
empty
Charlotte Führer
empty
empty
Edward Estlin Cummings
Jessie Laidlay Weston
empty
empty
empty
Mary Greene Ware
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Sarah Annie Frost
empty
empty
Mason Locke Weems
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Peter Hampson Ditchfield
Montague Rhodes James
empty
empty
empty
empty
empty
empty
Robert

empty
empty
Edith Nesbit
James Russell Miller
Catherine Grace Frances Gore
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Mary H. Maxwell
empty
empty
empty
empty
Annie Fellows Johnston
empty
empty
empty
empty
Ernest Alfred Wallis Budge
empty
empty
empty
empty
empty
empty
empty
empty
Edgar Albert Guest
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
William Henry Hudson
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Frank Lucius Packard
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Emanuel Haldeman-Julius
empty
St. John Greer Ervine
empty
empty
Clarence Edgar Edwords
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
Charlotte Mary Yonge
Mary Elizabeth Braddon
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
empty
David Herbert Lawrence
Da

,item,work,itemLabel,workLabel
0,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q8337,J. K. Rowling,Harry Potter
1,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q43361,J. K. Rowling,Harry Potter and the Philosopher's Stone
2,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q46751,J. K. Rowling,Harry Potter and the Goblet of Fire
3,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q46758,J. K. Rowling,Harry Potter and the Deathly Hallows
4,http://www.wikidata.org/entity/Q34660,http://www.wikidata.org/entity/Q46887,J. K. Rowling,Harry Potter and the Half-Blood Prince
...,...,...,...,...
0,http://www.wikidata.org/entity/Q433098,http://www.wikidata.org/entity/Q7739314,Charlotte Mary Yonge,The Heir of Redclyffe
0,http://www.wikidata.org/entity/Q433098,http://www.wikidata.org/entity/Q7739314,Charlotte Mary Yonge,The Heir of Redclyffe
0,http://www.wikidata.org/entity/Q213447,http://www.wikidata.org/entity/Q451517,John Millington Synge,The Playboy of the Western World
1,http://www.wikidata.org/entity/Q213447,http://www.wikidata.org/entity/Q3742640,John Millington Synge,Riders to the Sea


In [8]:
result_df.to_csv("authorM.csv")